In [16]:
import sys
sys.path.append('../..')
from pymongo import UpdateOne
import matplotlib.pyplot as plt
from exploration.config import mongo_inst
from pymongo import MongoClient
from mlpp.data_collection.sample import osuDumpSampler

In [17]:
client = MongoClient('localhost', 27017)

In [18]:
db = client.osu_random_db

In [19]:
data = mongo_inst["osu_random_db"]

In [20]:
test = osuDumpSampler(data)

In [21]:
test.use_random_sample("sample_scores_1M", 6500)

[11739434,
 8586020,
 18525523,
 16468324,
 13306892,
 18477420,
 14294434,
 16208425,
 13722439,
 15608883,
 11940805,
 14229663,
 17098696,
 15366556,
 16294180,
 15698556,
 18169891,
 11951122,
 14926487,
 17849788,
 13649783,
 18488326,
 18053572,
 14549796,
 2415309,
 19604926,
 17619126,
 17769778,
 14948944,
 16122158,
 15426454,
 11504360,
 14637457,
 14958906,
 13553856,
 17539310,
 14184499,
 10644003,
 15718672,
 6304150,
 16700063,
 13994569,
 4498162,
 11852932,
 17741290,
 6937512,
 17422353,
 14996823,
 18604096,
 17361101,
 19158250,
 18625398,
 5198536,
 15198191,
 9589018,
 16678913,
 11411990,
 14378050,
 17226402,
 15499818,
 1194825,
 12191805,
 13988532,
 14323014,
 6085073,
 11035144,
 2892977,
 19968777,
 15108998,
 4835292,
 16836876,
 9359589,
 11970092,
 17151877,
 15264156,
 15345269,
 4628750,
 9157668,
 15731026,
 14864953,
 11410769,
 16708402,
 14750379,
 9101071,
 11191790,
 18312735,
 15872699,
 17419368,
 12447866,
 17161512,
 7491235,
 10538801,
 175

In [22]:
data1 = mongo_inst["osu_random_db"]
test1 = osuDumpSampler(data1)

In [23]:
collection = data["sample_scores_1M"]

In [24]:
test.get_more_recent_than(collection, 2018, 12, 1, 0, 0, 0, "sample_scores_500k")

In [23]:
# test1.use_random_sample("sample_scores_100", 100)

In [25]:
collection2 = data["sample_scores_500k"]

In [28]:
"""
max est user pp in dataset of 500k
"""
cursor=db.sample_scores_500k.aggregate(
   [
     {
       "$group":
         {
           "_id": {},
           "max": { "$max": "$mlpp.est_user_pp" }
         }
     }
   ]
)
for document in cursor:
    print(document)
print(document['max'])
max_pp = document['max']

{'_id': {}, 'max': 10400.663273911368}
10400.663273911368


In [30]:
a = 0
b = 100

while b <= max_pp:
    db.uniform_collection2.insert_many(
        collection2.aggregate([
        {
        '$match': {
            'mlpp.est_user_pp' : {
                '$gt': a,
                '$lt': b,
            }
        }
    },
    {'$sample': {
        'size': 800
    }
}
        
])
    )
    a = b
    b += 100

InvalidOperation: No operations to execute

In [17]:
#l = [x * 100 for x in range (115)]
#print(l)

[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000, 6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200, 7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400, 8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600, 9700, 9800, 9900, 10000, 10100, 10200, 10300, 10400, 10500, 10600, 10700, 10800, 10900, 11000, 11100, 11200, 11300, 11400]


In [32]:
#once you have a uniform collection, use this to have a collection with the most 1000 popular maps:
db.uniform_collection.aggregate([
    {
        '$group': {
            '_id': '$beatmap_id', 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }, {
        '$limit': 1000
    }, {
        '$out': '1000_most_popular_maps2'
    }
])